In [333]:
import pandas as pd
from datetime import time

INPUT_DATA_FOLDER = "data"
WEATHER_FILE = "all_weather_by_hour.csv"
ALARMS_FILE = "alarms.csv"

import csv

with open(f"{INPUT_DATA_FOLDER}/{WEATHER_FILE}", newline='') as csvfile:
    weatherTemp = list(csv.reader(csvfile))
    
with open(f"{INPUT_DATA_FOLDER}/{ALARMS_FILE}", newline='') as csvfile:
    alarms = list(csv.reader(csvfile))[1:]
    
REGION_FILE = "regions.csv"
with open(f"{INPUT_DATA_FOLDER}/{REGION_FILE}", newline='') as csvfile:
    regions = list(csv.reader(csvfile))


In [301]:
# addition funcs
def findRegion(check):
    if check == "Хмельницька область":
        return '22'
    #  DO NOT TOUCH THIS AND DO NOT ASK WHY I DID THIS
    if check == ''.join(chr(65039)) + "Донеччина":
        return '5'
    for region in regions:
        for regionTemp in region:
            if regionTemp == check:
                return region[4]
    return None

def alarmsToDict(alarms):
    res = {}
    for alarm in alarms:
        temp = alarm[0].split(";")
        check1 = findRegion(temp[2].replace("\"", "").replace(" ", ""))
        check2 = findRegion(temp[3].replace("\"", "").replace(" ", ""))
        if check1 is None:
            check1 = check2
        try:
            res[check1]
        except (KeyError):
            res[check1] = []
        res[check1].append(pd.to_datetime(temp[5].replace("\"", "")))
        res[check1].append(pd.to_datetime(temp[6].replace("\"", "")))
    return res


    

In [311]:
alarmsDict = alarmsToDict(alarms)
for key in alarmsDict.keys():
    alarmsDict[key].sort()
    alarmsDict[key] = list(dict.fromkeys(alarmsDict[key]))

In [236]:
print(alarmsDict.keys())
print(alarmsDict['2'][:10])


dict_keys(['2', '13', '15', '6', '17', '3', '22', '19', '8', 5, '4', '11', '10', '25', '23', '16', '14', '24', '9', '20', '18', '7', '21'])
[Timestamp('2022-02-25 22:55:42'), Timestamp('2022-02-25 23:41:53'), Timestamp('2022-02-26 08:39:39'), Timestamp('2022-02-26 10:42:41'), Timestamp('2022-02-26 10:58:23'), Timestamp('2022-02-26 11:59:40'), Timestamp('2022-02-26 16:48:36'), Timestamp('2022-02-26 17:15:37'), Timestamp('2022-02-27 10:48:03'), Timestamp('2022-02-27 11:07:09')]


In [427]:
print(weatherTemp[0])

['city_latitude', 'city_longitude', 'city_resolvedAddress', 'city_address', 'city_timezone', 'city_tzoffset', 'day_datetime', 'day_datetimeEpoch', 'day_tempmax', 'day_tempmin', 'day_temp', 'day_feelslikemax', 'day_feelslikemin', 'day_feelslike', 'day_dew', 'day_humidity', 'day_precip', 'day_precipprob', 'day_precipcover', 'day_snow', 'day_snowdepth', 'day_windgust', 'day_windspeed', 'day_winddir', 'day_pressure', 'day_cloudcover', 'day_visibility', 'day_solarradiation', 'day_solarenergy', 'day_uvindex', 'day_severerisk', 'day_sunrise', 'day_sunriseEpoch', 'day_sunset', 'day_sunsetEpoch', 'day_moonphase', 'day_conditions', 'day_description', 'day_icon', 'day_source', 'day_preciptype', 'day_stations', 'hour_datetime', 'hour_datetimeEpoch', 'hour_temp', 'hour_feelslike', 'hour_humidity', 'hour_dew', 'hour_precip', 'hour_precipprob', 'hour_snow', 'hour_snowdepth', 'hour_preciptype', 'hour_windgust', 'hour_windspeed', 'hour_winddir', 'hour_pressure', 'hour_visibility', 'hour_cloudcover', 'h

In [434]:
weather = []
weatherDate = []
conditions = {}
icons = {}
for x in weatherTemp[1:]:
    temp = x.copy()
    temp.pop(0)
    temp.pop(0)
    temp.pop(1)
    temp.pop(1)
    temp.pop(1)
    for y in range(35):
        temp.pop(2)
    temp.pop(3)
    temp.pop(11)
    temp[0] = findRegion(temp[0].split(",")[0])
    weatherDate.append([temp[0], pd.Timestamp.combine(pd.to_datetime(temp[1]), pd.to_datetime(temp[2]).time())])
    temp.pop(1)
    temp.pop(1)
    temp.pop(21)
    temp.pop(21)
    try:
        conditions[temp[19]]
    except (KeyError):
        conditions[temp[19]] = len(conditions) + 1
    try:
        icons[temp[20]]
    except (KeyError):
        icons[temp[20]] = len(icons) + 1
    
    weather.append(temp.copy())


In [430]:
print(weather[15])
print(weatherDate[15])
print(conditions)
print(icons)


['23', '4.8', '3.4', '78.06', '1.3', '0.0', '0.0', '0.0', '0.0', '11.5', '6.1', '33.8', '1024.0', '0.5', '97.8', '140.0', '0.5', '1.0', '10.0', 'Overcast', 'fog']
['23', Timestamp('2022-02-24 15:00:00')]
{'Partially cloudy': 1, 'Overcast': 2, 'Clear': 3, 'Snow, Overcast': 4, 'Snow, Rain, Overcast': 5, 'Rain, Overcast': 6, 'Snow, Partially cloudy': 7, 'Rain, Partially cloudy': 8, 'Rain': 9, 'Snow, Rain, Partially cloudy': 10, 'Freezing Drizzle/Freezing Rain, Overcast': 11, 'Snow': 12, 'Snow, Rain': 13, 'Ice, Overcast': 14}
{'partly-cloudy-night': 1, 'cloudy': 2, 'fog': 3, 'clear-night': 4, 'clear-day': 5, 'partly-cloudy-day': 6, 'snow': 7, 'rain': 8, 'wind': 9}


In [386]:
def createAlarmsFromWeatherData(weather, alarmsDict):
    result = []
    for weatherLine in weather:
        started = False
        for data in alarmsDict[weatherLine[0]]:
            if data > (pd.Timestamp.combine(weatherLine[1], time(1, 0, 0))):
                if started:
                    result.append(1)
                else:
                    result.append(0)
                break
            started = not started
        
    return result

In [387]:
alarmsBit = createAlarmsFromWeatherData(weatherDate, alarmsDict)

In [389]:
print(alarmsBit[500:600])

[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [435]:
for x in weather:
    for i in range(len(x)):
        if i > 18:
            break
        if x[i] == '':
            x[i] = 0.0
        else:
            x[i] = float(x[i])
    x[19] = conditions[x[19]]
    x[20] = icons[x[20]]
   


In [450]:
print(len(weather))
print(len(alarmsBit))


190656
190656


In [457]:

import numpy
from sklearn import linear_model
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(weather, alarmsBit, test_size=0.20, shuffle=True)
                                                    
from sklearn.linear_model import LogisticRegression
log_reg_model = LogisticRegression(max_iter=2500000)

# Train (fit) the model
log_reg_model.fit(X_train, y_train)

# Make predictions
y_pred = log_reg_model.predict(X_test) # Predictions
y_true = y_test # True values

# Model evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

print("Accuracy:", np.round(accuracy_score(y_true, y_pred), 2))
precision, recall, fscore, _ = precision_recall_fscore_support(y_true, y_pred,
                                                               average='binary')

Accuracy: 0.69
